In [15]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score, ConfusionMatrixDisplay
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

In [2]:
df = pd.read_csv('dataset/movie.csv', sep=',')
df

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1
...,...,...
39995,"""Western Union"" is something of a forgotten cl...",1
39996,This movie is an incredible piece of work. It ...,1
39997,My wife and I watched this movie because we pl...,0
39998,"When I first watched Flatliners, I was amazed....",1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    40000 non-null  object
 1   label   40000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 625.1+ KB


#### 1. Data Pre-Processing: cleaning
text, remove stopwords,
stemming, lemmatization.**

Tahap ini melibatkan pra-pemrosesan data untuk membersihkan dan mengubah teks mentah menjadi bentuk yang dapat digunakan oleh SVM

In [ ]:
## Data cleaning

text = df['text']
# 1 Lowering case: Mengubah teks menjadi huruf kecil untuk konsistensi dalam analisis
text = text.str.lower()
# text

In [ ]:
# import re

# 2 Menghapus karakter khusus kecuali huruf dan angka
text = text.str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
# text

In [ ]:
# 3 Tokenisasi Memisahkan teks menjadi unit-unit yang lebih kecil seperti kata-kata atau frasa.
nltk.download('punkt')
text = text.apply(lambda x: word_tokenize(x))
# text

In [ ]:
# 4 Menghapus Stopwords Menghapus kata-kata umum yang tidak memberikan informasi penting dalam analisis.
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))  # Menggunakan kamus stop words Bahasa Inggris
text = text.apply(lambda x: [word for word in x if word not in stop_words])
text

In [ ]:
# 5 Menghapus Kata Pendek: Menghapus kata-kata dengan panjang karakter yang terlalu pendek yang cenderung tidak memiliki makna.

min_length = 3  # Menentukan panjang minimum kata yang diizinkan
text = text.apply(lambda x: [word for word in x if len(word) >= min_length])
# text


In [ ]:
# 6 Lematisasi atau Stemming: Mengubah kata-kata ke bentuk dasar mereka untuk mengurangi variasi kata yang sama
# Melakukan proses stemming (penggunaan kata dasar)
ps = PorterStemmer()
 
# Membersihkan kalimat dari kata yang ada di stopwords
review = text.apply(lambda x: [ps.stem(word) for word in x])

In [ ]:
review

In [ ]:
# 7 Menggabungkan Kembali Teks: Menggabungkan kembali unit-unit teks yang telah dibersihkan menjadi teks yang telah diolah sepenuhnya.
review = review.apply(lambda x: ' '.join(x))

In [ ]:
preprocess = pd.concat([review, df['label']], axis=1)
preprocess.to_csv('dataset/preprocess.csv',index=False)  
preprocess

#### 1.1 Load clean dataset

In [4]:
preprocess = pd.read_csv('dataset/preprocess.csv')

In [5]:
target_0 = preprocess[preprocess['label']==0].head(1000)
target_1 = preprocess[preprocess['label']==1].head(1000)
k_samples = pd.concat([target_0, target_1], axis=0)

#### 2. Feature Extraction: Bag-of-Word atau TF-IDF

Pada tahap ini, fitur-fitur atau representasi numerik diperoleh dari teks yang telah dipreproses

In [6]:
#Bag-of-Word
bow_cv = k_samples['text']
# Inisialisasi CountVectorizer
vectorizer = CountVectorizer()

# Memproses teks dan membangun vocabulary
bow_X = vectorizer.fit_transform(bow_cv)

# Membuat DataFrame dari hasil Bag-of-Words
df_bow = pd.DataFrame(bow_X.toarray(), columns=vectorizer.get_feature_names_out())

df_bow.shape


(2000, 22385)

In [7]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Inisialisasi TfidfVectorizer
vectorizer = TfidfVectorizer()

tfidf_text = k_samples['text']

# Memproses teks dan membangun TF-IDF
tfidf_X = vectorizer.fit_transform(tfidf_text)

# Membuat DataFrame dari hasil TF-IDF
df_tfidf = pd.DataFrame(tfidf_X.toarray(), columns=vectorizer.get_feature_names_out())

df_tfidf.shape

(2000, 22385)

#### 3. Pembagian Data

Data yang telah dipreproses dan direpresentasikan dalam bentuk fitur perlu dibagi menjadi set pelatihan (training set) dan set pengujian (test set). Set pelatihan digunakan untuk melatih model SVM, sedangkan set pengujian digunakan untuk menguji kinerja model

In [8]:
X = df_bow
y = k_samples['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### 4. Pelatihan Model

Model SVM dilatih menggunakan set pelatihan yang telah dibagi sebelumnya. SVM mempelajari pola dalam data pelatihan untuk memisahkan kelas positif dan negatif. Pada tahap ini, SVM menemukan hyperplane terbaik yang memaksimalkan margin antara kelas-kelas tersebut.

In [9]:
rfc = RandomForestClassifier()

In [10]:
rfc.fit(X_train, y_train)

RandomForestClassifier()

In [11]:
rfc_pred = rfc.predict(X_test)

In [25]:
lr = LogisticRegression(penalty='l2', max_iter=500, C=1, random_state=42)

In [26]:
lr.fit(X_train, y_train)

LogisticRegression(C=1, max_iter=500, random_state=42)

In [27]:
lr_pred = lr.predict(X_test)

#### 5. Evaluasi Model

Setelah model dilatih, langkah selanjutnya adalah menguji kinerjanya pada set pengujian yang telah dipisahkan sebelumnya. Beberapa metrik evaluasi yang umum digunakan dalam analisis sentimen adalah akurasi, presisi, recall, dan F1-score. Metrik ini membantu mengukur sejauh mana model dapat mengklasifikasikan sentimen dengan benar.

In [28]:
def evaluation(y_test, pred):
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    print("Precsion:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print(confusion_matrix(y_test, pred))
    print(classification_report(y_test, pred))

In [29]:
evaluation(y_test, rfc_pred)

Precsion: 0.8375634517766497
Recall: 0.8208955223880597
F1 Score: 0.8291457286432161
[[167  32]
 [ 36 165]]
              precision    recall  f1-score   support

           0       0.82      0.84      0.83       199
           1       0.84      0.82      0.83       201

    accuracy                           0.83       400
   macro avg       0.83      0.83      0.83       400
weighted avg       0.83      0.83      0.83       400



In [30]:
evaluation(y_test, lr_pred)

Precsion: 0.845360824742268
Recall: 0.8159203980099502
F1 Score: 0.8303797468354429
[[169  30]
 [ 37 164]]
              precision    recall  f1-score   support

           0       0.82      0.85      0.83       199
           1       0.85      0.82      0.83       201

    accuracy                           0.83       400
   macro avg       0.83      0.83      0.83       400
weighted avg       0.83      0.83      0.83       400



**6. Prediksi**

Setelah model dievaluasi dengan baik, model tersebut dapat digunakan untuk memprediksi sentimen pada data teks baru. Teks baru akan mengalami pra-pemrosesan yang sama seperti pada tahap 1 dan diubah menjadi fitur-fitur menggunakan metode yang sama seperti pada tahap 2. Kemudian, model SVM digunakan untuk mengklasifikasikan sentimen berdasarkan fitur-fitur tersebut

In [40]:
# Make predictions on new text data
input_text = ['this movie is bad to watch']
new_text_bow = vectorizer.transform(input_text)
y_pred = lr.predict(new_text_bow)

print("Predicted Sentiment:", y_pred[0])

Predicted Sentiment: 0
